In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.io import show, output_notebook
output_notebook()

from scripts_viz.visualization_utils import *
from scripts_viz.visualization_utils import TTQcolor

from bokeh.layouts import gridplot, row, column

Loading BokehJS ...

In [2]:
datafolder = '../data/viz_data/'
models = 'benchmarks_time_opt_'
postfix = 'p90'

experiment = models+postfix

viz = pd.read_csv(datafolder+experiment+'/'+experiment+'.csv')

In [3]:
pd.set_option("display.max_rows", 101)

In [4]:
viz.columns

Index(['Run ID', 'Name', 'Source Type', 'Source Name', 'User', 'Status',
       'alpha', 'average', 'bootstrap', 'class_weight', 'criterion',
       'early_stopping', 'epsilon', 'eta0', 'experiment_type', 'fit_intercept',
       'l1_ratio', 'learning_rate', 'loss', 'max_depth', 'max_features',
       'max_iter', 'max_leaf_nodes', 'min_impurity_decrease',
       'min_impurity_split', 'min_samples_leaf', 'min_samples_split',
       'min_weight_fraction_leaf', 'model_filename', 'model_filepath',
       'model_type', 'n_estimators', 'n_iter_no_change', 'n_jobs', 'oob_score',
       'penalty', 'power_t', 'random_state', 'shuffle', 'test_file_name',
       'test_file_path', 'test_size', 'tol', 'train_file_name',
       'train_file_path', 'train_size', 'validation_fraction', 'verbose',
       'warm_start', 'f_c_impaired1_c', 'f_c_lent_c', 'f_c_pastdue90_c',
       'f_c_repaid_c', 'f_c_trend_a', 'f_cd_impaired1_c', 'f_cd_lent_c',
       'f_cd_pastdue90_c', 'f_cd_repaid_c', 'f_cd_trend_a',
    

In [5]:
#viz.transpose()['model_filename'].str.split('.pkl')

In [6]:
def set_clf_cols(viz):
    
    viz = viz.copy()
    names = viz['model_filename']
    
    mod_names = []
    
    count=0
    postfix = ''
    for name in names:
        postfix = name.split(viz['model_type'][count])[1].replace(postfix+'__', '').replace('.pkl','')
        newname = viz['model_type'][count]+postfix
        mod_names.append(newname)
        count+=1
    
    viz.index = mod_names
    
    return viz.transpose()
    

In [7]:
viz = set_clf_cols(viz)

In [8]:
viz.columns

Index(['RandomForestClassifier_190812_74557',
       'RandomForestClassifier_190812_74540',
       'RandomForestClassifier_190812_74520',
       'RandomForestClassifier_190812_74457',
       'RandomForestClassifier_190812_74414',
       'RandomForestClassifier_190812_74352', 'SGDClassifier_190812_74258',
       'SGDClassifier_190812_74252', 'SGDClassifier_190812_74246',
       'SGDClassifier_190812_74237', 'SGDClassifier_190812_74228'],
      dtype='object')

In [9]:
files = []
for modelname in viz.columns:
    check = viz.loc['model_filepath', modelname].replace('models', 'viz_data').replace('.pkl','_viz.pkl')
    if type(check) is str:
        files.append(check)
    else:
        string = check.values[0]
        if string not in set(files):
            files.append(string)
print(len(files))
        

11


In [10]:
viz

,RandomForestClassifier_190812_74557,RandomForestClassifier_190812_74540,RandomForestClassifier_190812_74520,RandomForestClassifier_190812_74457,RandomForestClassifier_190812_74414,RandomForestClassifier_190812_74352,SGDClassifier_190812_74258,SGDClassifier_190812_74252,SGDClassifier_190812_74246,SGDClassifier_190812_74237,SGDClassifier_190812_74228
Run ID,2043936070cc4e8ca757c877b0fae714,a26343bc3ef24ed78e590649f1766cf6,8bda0cd104444a13ad64060551bfc5b4,97448a9d852b41dc8e958e2e4cae91e5,9ca20523f8414c1b9bde0bc84f91776b,742abcd7a25246d5926b75094e58dd95,b3bb99fdc4d14ab0a4255355cd94683e,e802ba0b8f0246d7a77a735d3179f77e,c43730e1779e47018ccc286a01430e21,b171406dd4be4c039ec5b3cc72fa16fe,34b49bda96f545d9b6549079d96789e5
Name,time_2018-04-30_p90_RandomForestClassifier,time_2018-04-30_p90_RandomForestClassifier,time_2018-04-30_p90_RandomForestClassifier,time_2018-04-30_p90_RandomForestClassifier,time_2018-04-30_p90_RandomForestClassifier,time_2018-04-30_p90_RandomForestClassifier,time_2018-04-30_p90_SGDClassifier,time_2018-04-30_p90_SGDClassifier,time_2018-04-30_p90_SGDClassifier,time_2018-04-30_p90_SGDClassifier,time_2018-04-30_p90_SGDClassifier
Source Type,LOCAL,LOCAL,LOCAL,LOCAL,LOCAL,LOCAL,LOCAL,LOCAL,LOCAL,LOCAL,LOCAL
Source Name,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...,C:\ProgramData\Anaconda3\envs\thesis_mlflow\li...
User,david,david,david,david,david,david,david,david,david,david,david
Status,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED
alpha,NaN,NaN,NaN,NaN,NaN,NaN,0.0001,0.0001,0.0001,0.0001,0.0001
average,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False
bootstrap,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
class_weight,balanced,balanced,balanced,balanced,balanced,balanced,None,None,None,None,None


In [11]:
files

['../data/viz_data/benchmarks_time_opt_p90/time_2018-04-30_p90__RandomForestClassifier_190812_74557_viz.pkl',
 '../data/viz_data/benchmarks_time_opt_p90/time_2018-04-30_p90__RandomForestClassifier_190812_74540_viz.pkl',
 '../data/viz_data/benchmarks_time_opt_p90/time_2018-04-30_p90__RandomForestClassifier_190812_74520_viz.pkl',
 '../data/viz_data/benchmarks_time_opt_p90/time_2018-04-30_p90__RandomForestClassifier_190812_74457_viz.pkl',
 '../data/viz_data/benchmarks_time_opt_p90/time_2018-04-30_p90__RandomForestClassifier_190812_74414_viz.pkl',
 '../data/viz_data/benchmarks_time_opt_p90/time_2018-04-30_p90__RandomForestClassifier_190812_74352_viz.pkl',
 '../data/viz_data/benchmarks_time_opt_p90/time_2018-04-30_p90__SGDClassifier_190812_74258_viz.pkl',
 '../data/viz_data/benchmarks_time_opt_p90/time_2018-04-30_p90__SGDClassifier_190812_74252_viz.pkl',
 '../data/viz_data/benchmarks_time_opt_p90/time_2018-04-30_p90__SGDClassifier_190812_74246_viz.pkl',
 '../data/viz_data/benchmarks_time_op

In [12]:
#retrieving dict viz and organizing it in a macrodictionary
viz_dicts = dict(zip(list(viz.columns), [pd.read_pickle(viz_data) for viz_data in files]))

In [13]:
TTQcolor.keys()

dict_keys(['font', 'lightGrey', 'borderColour', 'cream', 'lightCream', 'background', 'link', 'brightLink', 'marketplaceOrange', 'warningRed', 'Salmon', 'victorian', 'whiteGrey', 'eightyGrey', 'blueGrey', 'sixtyGrey', 'navy', 'darkPurple', 'darkNavy', 'darkCyan', 'redBrown', 'richBrown', 'algae', 'mutedBlue', 'azureBlue', 'electric', 'sky', 'turq', 'pea', 'ocean', 'richOrange', 'richPeach', 'yell', 'yellowOrange', 'peach', 'bloodRed', 'PPTbg'])

In [14]:
colors = [TTQcolor['azureBlue'], TTQcolor['richOrange'], TTQcolor['algae'],
                                                       TTQcolor['yell'], TTQcolor['redBrown'], TTQcolor['bloodRed']]

In [15]:
val_roc = plot_rocs([viz_dicts[model]['validation'] for model in viz.columns if 'RandomForest' in model], label = list(viz.columns), title_lab = 'Validation performance', 
               p_width=600, p_height=600, line_width=2,
                    colors = colors, legend_font_size='9pt', fpr_font_size='9pt',
                   bestFprOnly=True, show_legend=False)

show(val_roc)

In [ ]:
test_roc = plot_rocs([viz_dicts[model]['testing'] for model in viz.columns if 'RandomForest' in model], label = list(viz.columns), title_lab = 'Test performance',
               p_width=600, p_height=600, line_width=2,
                     colors = colors, legend_font_size='9pt', fpr_font_size='9pt',
                    bestFprOnly=True, show_legend=False)

show(test_roc)

In [17]:
viz.loc['tp_rate'] = viz.loc['test_tp']/(viz.loc['test_tp'] + viz.loc['test_fn'])
viz.loc['tn_rate'] = viz.loc['test_tn']/(viz.loc['test_tn'] + viz.loc['test_fp'])
viz.loc['fp_rate'] = viz.loc['test_fp']/(viz.loc['test_tp'] + viz.loc['test_tn'])
viz.loc['fn_rate'] = viz.loc['test_fn']/(viz.loc['test_tn'] + viz.loc['test_tp'])

In [18]:
params = ['tp_rate', 'tn_rate', 'fp_rate', 'fn_rate', 'test_auc', 'val_auc']
models = [m for m in list(viz.columns) if 'RandomForest' in m]
spider = spiderWebChart(models, params, 
                    [viz.loc[params, m] for m in models], 
                    colors=colors, text_size='12pt',
                       title='Overall comparison', p_height=600, p_width=600, margin_distance=0.25,
                       legend_location='top_right', show_legend=False, line_width=4.5, fill_alpha=0.1)

show(spider)

In [19]:
params = ['tp_rate', 'tn_rate', 'fp_rate', 'fn_rate', 'test_auc', 'val_auc']
models = [m for m in list(viz.columns) if 'RandomForest' in m]

cols = []
rows = []

for m in range(len(models)):
    if m%3==0 or m==len(models):
        rows = []
        cols.append(rows)
        
    single_spider = spiderWebChart([models[m]], params, 
                            [viz.loc[params, models[m]]], 
                            colors=[colors[m]], text_size='6pt',
                               title='', p_height=200, p_width=200, margin_distance=0.25,
                               legend_location='top_right', show_legend=False, line_width=3.5, fill_alpha=0.1)
    
    rows.append(single_spider)
    
l_sp = gridplot(cols)

show(l_sp)

In [20]:
f = [fold for fold in viz.index if 'val_auc_fold_' in fold]
sorter = list(pd.Series([fold for fold in viz.index if 'val_auc_fold_' in fold]).apply(lambda x:int(x.split('_')[-1])))
              
folds_name = list(pd.Series(sorted(list(zip(f,sorter)), key=lambda f:f[1])).apply(lambda x:x[0]))

In [21]:
folds = histfolds([m for m in list(viz.columns) if 'RandomForest' in m], folds_name, viz, plot_w=1200, title="Validation folds performance",
                 colors=colors, xlabelorientation=1.55, group_text_font_size='6pt')

show(folds)

In [22]:
folds_name

['val_auc_fold_1',
 'val_auc_fold_3',
 'val_auc_fold_4',
 'val_auc_fold_5',
 'val_auc_fold_6',
 'val_auc_fold_7',
 'val_auc_fold_8',
 'val_auc_fold_9',
 'val_auc_fold_10',
 'val_auc_fold_11',
 'val_auc_fold_12']

In [23]:
f = [fold for fold in viz.index if 'val_auc_fold_' in fold]
sorter = list(pd.Series([fold for fold in viz.index if 'val_auc_fold_' in fold]).apply(lambda x:int(x.split('_')[-1])))
              
folds_name = list(pd.Series(sorted(list(zip(f,sorter)), key=lambda f:f[1])).apply(lambda x:x[0]))

cols = []
rows = []

for m in range(3, len(models)+1, 3):
    if m%3==0 or m==len(models):
        rows = []
        cols.append(rows)
        
    folds = histfolds(models[m-3:m], folds_name, viz, plot_w=600, title="Validation folds performance",
                 colors=colors[m-3:m], xlabelorientation=1.55, group_text_font_size='6pt')
    
    rows.append(folds)
    
ff_sp = gridplot(cols)

show(ff_sp)

In [24]:
ss = modelSpreadsheet(viz, ['n_estimators', 'max_depth', 'max_features', 'max_leaf_nodes', 
               'min_samples_leaf', 'min_samples_split', 'bootstrap',
              'criterion', 'val_auc', 'test_auc'], 'RandomForestClassifier', color_cells=True, colors=colors,
                      index_header='RF', height=180, width=1200, index_width=25,
                     row_height=25)

show(ss)

In [25]:
from bokeh.io import show
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure

features = list(viz.index[list(pd.Series(viz.index).str.contains('^f_', regex=True))])
models = [c for c in list(viz.columns) if 'RandomForest' in c]


mod_dict = {}
for m in models:
    val_list = []
    for f in features:
        val_list.append(viz.loc[f, m])
    mod_dict[m] = val_list 
        
data = {**{'features': features}, **mod_dict}

x=[]
colors_list = []

for feat in features:
    count=0
    for mod in models:
        x.append((feat, mod))
        colors_list.append(colors[count])
        count+=1

counts = []
for featval in range(len(data[models[0]])):
    for model in models:
        counts.append(data[model][featval])

source = ColumnDataSource(data=dict(x=x, counts=counts, color=colors_list))

p = figure(x_range=FactorRange(*x), plot_width=1200, plot_height=500, title="",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source, color='color')

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1.55
p.xaxis.major_tick_line_color = None
p.xaxis.major_label_text_font_size = '0pt'
p.xaxis.group_text_font_size = '10pt'
p.xaxis.group_label_orientation = 1.57
p.xgrid.grid_line_color = None

show(p)

## Gridplot

In [28]:
#from bokeh.io import output_file
#output_file('try_viz.html')

In [29]:
l = gridplot([[ss],
              [row(val_roc, ff_sp)],
              [row(test_roc, l_sp)],[p]])
              #[row(val_roc, test_roc)]]) #,[row(val_roc, test_roc)],[folds], [spider]])
show(l)